#### Aaron Kaminsky

#### AJK259

#### CS 4701: Practicum in Artificial Intelligence

#### Presentation: 5/18/2021 7:30 PM - 8:30 PM

# Predicting the Outcome of NBA Games Through the Use of a Feedforward Neural Network

This project will consist of the implementation for a Feedforward Neural Network created to predict the outcome of National Basketball Association matchups. Every single Artificial Intelligence project starts with the collection of a corpus. For this project, National Basketball Association seasonal data was collected from Basketball-Reference.com. The data focused on the matchups throughout a basketball season and included data points such as the date, score and whether the game went into overtime. The corpus consists of the 2016-17, 2017-18, 2018-19 and 2019-20 NBA seasonal data. The most important datapoints for this project will consist of the date, team names and score. 




In [ ]:
from google.colab import drive
import os
import pandas as pd

drive.mount('/content/drive', force_remount=True)

# Obtaining the NBA Corpus Data
file_names = ["NBA_data_2019-20.csv", "NBA_data_2018-19.csv", "NBA_data_2017-18.csv", "NBA_data_2016-17.csv"]
data_path = os.path.join(os.getcwd(), "drive", "My Drive", "CS 4701") 

data_location = os.path.join(data_path, file_names[0])
csv_data = pd.read_csv(data_location)

# Imported Classes
After the corpus has been collected and studied, the next step is to determine the tools needed to implement the project. The libraries used to assist this implementation consist of random and torch. Random is a library that allows user to use different functions revolving around randomness. For this project, random will be used to shuffle a list of data to randomly select a portion of test data and a portion of validation data. Torch is a library that focuses on deep learning and provides tools to create neural networks. Torch is used throughout this project to format the data and create the feedforward neural network. 



In [2]:
import random
import torch
import torch.nn as nn
from torch.nn import init
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from tqdm.notebook import tqdm, trange

# Processing NBA Data
Once the libraries have been installed, the next step is to prepare the data to be utilized by the implemented neural network. In order to do this, a class was created representing an NBA game. This class collected the name, seasonal wins, and current win streak for each team in the matchup. These datapoints are important as they will be the inputs for the neural network. 

**The data for this project was collected on [Basketball-Reference.com](https://www.basketball-reference.com/leagues/)**


In [3]:
# This class is the representation of a singular NBA game
class NBA_game:
  def __init__(self, row_tuple, home_history, away_history, home_wins, away_wins):
    raw_data = row_tuple[1]

    self.away = raw_data[2] #Away team name
    self.home = raw_data[4] #Home team name
    self.home_streak = home_history #The Home teams win streak
    self.home_wins = home_wins #The number of wins by the home team
    self.away_wins = away_wins #The number of wins by the away team
    self.away_streak = away_history #The Away team's win streak
    
    self.win = 0
    if raw_data[3] < raw_data[5]:
      self.win = 1
    
    self.OT = 0
    if raw_data[7] == "OT":
      self.OT = 1

# create_NBA_games: takes raw_data and returns a list of NBA_Game classes
def create_NBA_games(raw_data):
  list_games = [] # List containing the NBA_game class data for each game in raw_data
  team_history = {} # Dictionary containing the win streak for each team
  team_wins = {} # Dinctionary containing the win total for each team
  team_idx = {} # The map representing team to index
  
  for game in raw_data.iterrows():
    home_history = 0
    away_history = 0
    home_wins = 0
    away_wins = 0
    home_team = game[1][4]
    away_team = game[1][2]
    
    if home_team in team_history:
      home_history = team_history[home_team]
      home_wins = team_wins[home_team]
    else:
      team_idx[home_team] = len(team_idx)
    if away_team in team_history:
      away_history = team_history[away_team]
      away_wins = team_wins[away_team]
    else:
      team_idx[away_team] = len(team_idx)
    
    game_data = NBA_game(game, home_history, away_history, home_wins, away_wins)
    list_games.append(game_data)
    
    if game_data.win == 1:
      team_history[home_team] = game_data.home_streak + 1
      team_wins[home_team] = game_data.home_wins + 1
      team_history[away_team] = 0
      team_wins[away_team] = game_data.away_wins
    else:
      team_history[home_team] = 0
      team_wins[home_team] = game_data.home_wins
      team_history[away_team] = game_data.away_streak + 1
      team_wins[away_team] = game_data.away_wins + 1
  return list_games, team_idx

After the NBA game data is created and stored in a list, the Corpus is randomly shuffled and split into a list of training data used to train the Neural Networks and a list of testing data used to evaluate the accuracy. Of the entire corpus, 90 percent is utilized to train the models and 10 percent is utilized to validate the models. 

In [4]:
NBA_game_data = [] # A list of NBA_game classes for the entire Corpus

for file_name in file_names:
  data_location = os.path.join(data_path, file_name)
  csv_data = pd.read_csv(data_location)
  season_data, team2idx = create_NBA_games(csv_data)
  NBA_game_data = NBA_game_data + season_data

random.shuffle(NBA_game_data)
split_num = round(len(NBA_game_data)/10)
train_data = NBA_game_data[split_num:] # The data used to train the models
test_data = NBA_game_data[:split_num] # The data used to test the accuracy of the models

# Feedforward Neural Network Implementation
In order to start the implementation, the data needs to be converted into a tensor, which is used by torch. A tensor is similar to a matrix, which contains a series of numbers that represent specific features. The first challenge was to convert the team names from a string into a number. This was completed through the use of a dictionary that mapped individual team names into a specific index. After the mapping is implemented, the tensor can be easily created as the seasonal wins, and win streak are both integers. The next step is to create the neural network model as well as the training and evaluation functions that will allow the model to learn. 

In [5]:
# ffnn_convert_to_vector takes a List of NBA_game objects and creates a vector used by the Feedforward Neural Network
def ffnn_convert_to_vector( data, team2idx ):
  vectorized_data = []
  
  for game in data:
    torch_data = torch.zeros(6)
    torch_data[0] += team2idx[game.home]
    torch_data[1] += game.home_wins
    torch_data[2] += game.home_streak
    torch_data[3] += team2idx[game.away]
    torch_data[4] += game.away_wins
    torch_data[5] += game.away_streak
    
    vectorized_data.append((torch_data, game.win))
  return vectorized_data

In [6]:
"""
The following classes and functions were taken from a previous project in CS 4740: 
Introduction to Natural Language Processing and edited to process NBA game data 
instead of detecting Emotion in tweets. 
"""
# NBADataset is a PyTorch class used to interact with the NBA_game class
class NBADataset(Dataset):
    def __init__(self, data):
        self.X = torch.cat([X.unsqueeze(0) for X, _ in data])
        self.y = torch.LongTensor([y for _, y in data])
        self.len = len(data)
    
    def __len__(self):
        return self.len
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]

# ffnn_data_loaders creates the data loaders used by the ffnn model
def ffnn_data_loaders(train, test, batch_size):
    dataset = NBADataset(train + test)

    train_indices = [i for i in range(len(train))]
    test_indices = [i for i in range(len(train), len(train) + len(test))]

    train_sampler = SubsetRandomSampler(train_indices)
    train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
    
    test_sampler = SubsetRandomSampler(test_indices)
    test_loader = DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)

    return train_loader, test_loader

# FFNN is a PyTorch class that represents a Feedforward Neural Network
class FFNN(nn.Module):
	def __init__(self, input_dim, h, output_dim):
		super(FFNN, self).__init__()
		self.h = h
		self.W1 = nn.Linear(input_dim, h) 
		self.activation = nn.ReLU()
		self.W2 = nn.Linear(h, output_dim)
	
		self.normalize = nn.GroupNorm(4, input_dim) 
	
		self.softmax = nn.LogSoftmax(dim=1) 
		self.loss = nn.NLLLoss() 

	def compute_Loss(self, predicted_vector, gold_label):
		return self.loss(predicted_vector, gold_label)

	def forward(self, input_vector):
		z0=self.activation(input_vector) 
		z1 = self.W1(z0)
		z2 = self.W2(z1)
		predicted_vector = self.softmax(z2)
		return predicted_vector
	
	def load_model(self, save_path):
		self.load_state_dict(torch.load(save_path))
	
	def save_model(self, save_path):
		torch.save(self.state_dict(), save_path)
  
# train_epoch trains the model using the training data loader
def train_epoch(model, train_loader, optimizer, print_res):
	model.train()
	total = 0
	loss = 0
	correct = 0
	
	for (input_batch, expected_out) in tqdm(train_loader, leave=False, desc="Training Batches"):
		output = model(input_batch.to(get_device()))
		total += output.size()[0]
		_, predicted = torch.max(output, 1)
		correct += (expected_out == predicted.to("cpu")).cpu().numpy().sum()
		loss = model.compute_Loss(output, expected_out.to(get_device()))
		optimizer.zero_grad()
		loss.backward()
		optimizer.step()
	
	if print_res:
	  print("========================\nTrain_Epoch \tPercent Correct = " + str(correct / total))
	return

# evaluation utilizes test_loader to validate the accuracy of model
def evaluation(model, test_loader, optimizer, print_res):
	model.eval()
	loss = 0
	correct = 0
	total = 0
	
	for (input_batch, expected_out) in tqdm(test_loader, leave=False, desc="Validation Batches"):
		output = model(input_batch.to(get_device()))
		total += output.size()[0]
		_, predicted = torch.max(output, 1)
		correct += (expected_out.to("cpu") == predicted.to("cpu")).cpu().numpy().sum()
		loss += model.compute_Loss(output, expected_out.to(get_device()))
	loss /= len(test_loader)
	
	if print_res:
	  print("Validation Metrics (Evaluation)\tPercent Correct = " + str(correct / total) + "\n========================\n")
	pass

# train_and_evaluate trains and evaluates the ffnn model
def train_and_evaluate(number_of_epochs, model, train_loader, test_loader, print_res):
	optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
	epoch_num = 1
	
	for epoch in trange(number_of_epochs, desc="Epochs"):
		if epoch_num == number_of_epochs:
		  print_res = True
		
		if print_res:
		  print("Epoch number: ", epoch_num) 
		
		train_epoch(model, train_loader, optimizer, print_res)
		evaluation(model, test_loader, optimizer, print_res)
		epoch_num += 1
	return model

Once the model was created, there were three hyperparameters that could be tuned to adjust the performance of the neural network. These hyperparameters affect the interactions between the neurons (h), the number of datapoints tested before updating the model (batch size), and the number of times the model will train using the same data (number of epochs). The best way to tune these hyperparameters is the run an experiment to determine how increasing these values will affect the overall performance of the system. 

In [11]:
get_device = lambda : "cuda:0" if torch.cuda.is_available() else "cpu"

ffnn_train_vectorized = ffnn_convert_to_vector(train_data, team2idx)
ffnn_test_vectorized = ffnn_convert_to_vector(test_data, team2idx)

# run_ffnn_model will run the Feedforward Neural Network based on certain parameters
def run_ffnn_model(batch_num, h, num_epoch, print_all = False):
  ffnn_train_loader, ffnn_test_loader = ffnn_data_loaders(ffnn_train_vectorized, ffnn_test_vectorized, batch_size=batch_num)
  model = FFNN(6, h, 2).to(get_device())
  return train_and_evaluate(num_epoch, model, ffnn_train_loader, ffnn_test_loader, print_all)

batch_size = 16
h = 512
num_epochs = 6
_ = run_ffnn_model(batch_size, h, num_epochs, print_all= True)

Epoch number:  1


Train_Epoch 	Percent Correct = 0.5711471103327496


Validation Metrics (Evaluation)	Percent Correct = 0.6023622047244095

Epoch number:  2


Train_Epoch 	Percent Correct = 0.5564798598949212


Validation Metrics (Evaluation)	Percent Correct = 0.6141732283464567

Epoch number:  3


...

Validation Metrics (Evaluation)	Percent Correct = 0.5511811023622047

Epoch number:  5


Train_Epoch 	Percent Correct = 0.5709281961471103


Validation Metrics (Evaluation)	Percent Correct = 0.6122047244094488

Epoch number:  6


Train_Epoch 	Percent Correct = 0.5777145359019265


Validation Metrics (Evaluation)	Percent Correct = 0.6456692913385826




# Testing The Effects of Hyperparameters on The Performance of The FeedForward Neural Network

The Hyperparameters used for the Feedforward Neural Network are the following:
* batch_size: This represents the number of data entries that are contained in each training and validation batch.
* num_Epoch: This represents the number of times the model is  trained and validated. 
* h: This represents the hidden dimensionality of the Neural Network. 

## 1. Testing the effects of batch_size
While keeping the number of epochs constant at six and the h value constant at 512, the following code will train and validate the same data using different batch sizes. The batch sizes and the average validation results are as follows:

1. batch_size = 1: 62.53% correct. 
2. batch_size = 4: 57.55% correct.
3. batch_size = 16: 60.37% correct. 
4. batch_size = 64: 52.89% correct. 
5. batch_size = 256: 58.92% correct. 

The data from the experiment shows that as the batch size increases, the accuracy of the model decreases. This results from the neural network being updated every batch_size amount instead of every data value. This results in a decrease in accuracy and runtime.

In [14]:
batch_range = [1, 4, 16, 64, 256]
h = 512
num_epochs = 6

for batch_size in batch_range:
  print("Batch Size = ", batch_size)
  ffnn_config = run_ffnn_model(batch_size, h, num_epochs)

Batch Size =  1


Epoch number:  6


Train_Epoch 	Percent Correct = 0.5737740805604203


Validation Metrics (Evaluation)	Percent Correct = 0.6141732283464567


Batch Size =  4


Epoch number:  6


...

Train_Epoch 	Percent Correct = 0.5501313485113836


Validation Metrics (Evaluation)	Percent Correct = 0.5275590551181102


Batch Size =  256


Epoch number:  6


Train_Epoch 	Percent Correct = 0.5785901926444834


Validation Metrics (Evaluation)	Percent Correct = 0.5885826771653543




## 2. Testing the effects of the number of epochs
While keeping the batch size at a constant value of 16 and the h value at a constant value of 512, the following code will test how to accuracy of the model is affected by the number of epochs. The batch sizes and average validation results are as follows:

1. num_epochs = 1: 54.46% correct.
2. num_epochs = 4: 52.17% correct.
3. num_epochs = 16: 61.16% correct.
4. num_epochs = 32: 62.99% correct.
5. num_epochs = 64: 64.04% correct.

The data from the experiment shows that as the number of epochs increase, the accuracy of the model increases as well. This is a direct result of the model having the ability to use the training data more as the number of epochs increase. 

In [13]:
batch_size = 16
h = 512
num_epochs_range = [1, 4, 16, 32, 64]

for num_epochs in num_epochs_range:
  print("num_epochs = ", num_epochs)
  ffnn_config = run_ffnn_model(batch_size, h, num_epochs)

num_epochs =  1


Epoch number:  1


Train_Epoch 	Percent Correct = 0.556260945709282


Validation Metrics (Evaluation)	Percent Correct = 0.5984251968503937


num_epochs =  4


Epoch number:  4


...

Train_Epoch 	Percent Correct = 0.6059544658493871


Validation Metrics (Evaluation)	Percent Correct = 0.5275590551181102


num_epochs =  64


Epoch number:  64


Train_Epoch 	Percent Correct = 0.632661996497373


Validation Metrics (Evaluation)	Percent Correct = 0.6377952755905512




## 3. Testing the effects of the h value
While keeping the number of epochs constant at six and the batch size constant at sixteen, the following code will train and validate the same data using different h values. The h values and the average validation results are as follows:

1. h = 1: 64.81% correct.
2. h = 32: 63.73% correct.
3. h = 128: 61.32% correct.
4. h = 512: 58.71% correct.
5. h = 4096: 54.09% correct.

The data from the experiment shows that there is a range of h values that will maximize the accuracy of the results; however, outside of this range there are too little or too many neurons that will negatively affect the performance of the model. 

In [12]:
batch_size = 16
h_range = [1, 32, 128, 512, 4096]
num_epochs = 6

for h in h_range:
  print("h = ", h)
  ffnn_config = run_ffnn_model(batch_size, h, num_epochs)

h =  1


Epoch number:  6


Train_Epoch 	Percent Correct = 0.6359457092819615


Validation Metrics (Evaluation)	Percent Correct = 0.6535433070866141


h =  32


Epoch number:  6


...

Train_Epoch 	Percent Correct = 0.5720227670753065


Validation Metrics (Evaluation)	Percent Correct = 0.5078740157480315


h =  4096


Epoch number:  6


Train_Epoch 	Percent Correct = 0.5663309982486865


Validation Metrics (Evaluation)	Percent Correct = 0.5452755905511811




# Refinements

Based on the performance of this implementation of a Feedforwarding Neural Network, there are a few refinements that can be made in order to improve the performance of the model. The biggest limiting factor for the model is the number of inputs provided. The inputs provided for each NBA team are the team name, team win streak and number of wins on the season. Some inputs that can potentially improve the performance could include player data, the projected starting court and coaching metrics. Similarly, utilizing a corpus containing the entire history of the National Basketball Association would allow for the model to increase its accuracy through increased training. In addition to increasing the complexity of the inputs, having a faster computer would allow the model to be run with significantly low batch sizes and high number of epochs. This would tremendously increase the performance of the model through increase training and validation, while also slightly increasing runtime as a result of better hardware. Lastly, the performance of this model can be improved through the use of a difference type of neural network. Feedforward is a basic type of neural network, so a more complex model would help to increase the accuracy of predicting a basketball game.